In [16]:
import duckdb
import pandas as pd
import polars as pl

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb

ModuleNotFoundError: No module named 'polars'

In [ ]:
%%sql
create or replace table df as
select *
from read_csv_auto('C:/Users/Guada/Desktop/dmeyf/tp_final/data/competencia_01_crudo.csv')

,Count
0,978439


In [ ]:
# Conecto a DuckDB en memoria
#con = duckdb.connect()

# Registro el DataFrame Pandas df como una tabla en DuckDB
#con.register("df", df)

In [3]:
%sql select * from df limit 100

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,0.0,4,0.00,-20833.47,0.00,2814,6743.38,6,0,9559.95
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,0.0,11,13267.49,-9662.51,0.00,5541,9419.90,8,0,1900.26
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,0.0,4,0.00,-4692.00,0.00,577,2821.32,2,0,6885.51
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,0.0,25,23800.17,-20349.22,0.00,7916,23509.71,8,0,1219.92
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,0.0,4,0.00,-88178.37,0.00,1958,20397.54,8,0,2627.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,249651085,202101,1,0,0,45,32,1045.40,8418.22,527.76,...,0.0,4,0.00,-28245.84,0.00,952,15677.88,9,0,5724.24
96,249654219,202101,1,0,0,59,305,2348.19,10505.68,1894.82,...,0.0,4,0.00,-474.63,0.00,7444,470.23,2,0,23.46
97,249655022,202101,1,0,0,55,157,5879.35,82470.18,3902.82,...,0.0,4,0.00,-70380.00,0.00,3603,67196.93,6,0,20339.82
98,249660786,202101,1,0,0,67,307,2450.81,25228.39,2301.36,...,0.0,4,0.00,-13798.51,27.43,8945,7040.66,5,0,4304.91


In [4]:
%sql select count(distinct(numero_de_cliente)) from df

,count(DISTINCT numero_de_cliente)
0,168881


In [5]:
%sql select distinct(foto_mes) from df

,foto_mes
0,202106
1,202104
2,202103
3,202105
4,202102
5,202101


In [7]:
%%time
duckdb.query("""
    SELECT 
        numero_de_cliente,
        SUM(CASE WHEN foto_mes = 202101 THEN 1 ELSE 0 END) AS '202101',
        SUM(CASE WHEN foto_mes = 202102 THEN 1 ELSE 0 END) AS '202102',
        SUM(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS '202103',
        SUM(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS '202104',
        SUM(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS '202105',
        SUM(CASE WHEN foto_mes = 202106 THEN 1 ELSE 0 END) AS '202106'
    FROM df
    GROUP BY numero_de_cliente
    ORDER BY numero_de_cliente
""").df().tail(150)

CPU times: total: 500 ms
Wall time: 153 ms


,numero_de_cliente,202101,202102,202103,202104,202105,202106
168731,1593333841,0.0,0.0,0.0,0.0,0.0,1.0
168732,1593346531,0.0,0.0,0.0,0.0,1.0,1.0
168733,1593384824,0.0,0.0,0.0,0.0,1.0,1.0
168734,1593419025,0.0,0.0,0.0,0.0,0.0,1.0
168735,1593419093,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
168876,1598368433,0.0,0.0,0.0,0.0,0.0,1.0
168877,1598419415,0.0,0.0,0.0,0.0,0.0,1.0
168878,1598425905,0.0,0.0,0.0,0.0,0.0,1.0
168879,1598470389,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# DF que contiene id de cliente, mes en fotmato 0-1 y cantidad de meses total

# Traigo todos los valores distintos de foto_mes
meses = duckdb.query("SELECT DISTINCT foto_mes FROM df ORDER BY foto_mes").df()['foto_mes'].tolist()

#Itero sobre los meses para crear el case - when
cols = ",\n".join([f"SUM(CASE WHEN foto_mes = {m} THEN 1 ELSE 0 END) AS \"{m}\"" for m in meses])

#Armo columnas indicando los meses en que está cada persona
str_meses = f"""
    SELECT 
        numero_de_cliente,
        {cols}, count(foto_mes) as q_meses
    FROM df
    GROUP BY numero_de_cliente
    ORDER BY numero_de_cliente
"""

df_meses = duckdb.query(str_meses).df()

df_meses.tail(150) 


CPU times: total: 719 ms
Wall time: 276 ms


,numero_de_cliente,202101,202102,202103,202104,202105,202106,q_meses
168731,1593333841,0.0,0.0,0.0,0.0,0.0,1.0,1
168732,1593346531,0.0,0.0,0.0,0.0,1.0,1.0,2
168733,1593384824,0.0,0.0,0.0,0.0,1.0,1.0,2
168734,1593419025,0.0,0.0,0.0,0.0,0.0,1.0,1
168735,1593419093,0.0,0.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...
168876,1598368433,0.0,0.0,0.0,0.0,0.0,1.0,1
168877,1598419415,0.0,0.0,0.0,0.0,0.0,1.0,1
168878,1598425905,0.0,0.0,0.0,0.0,0.0,1.0,1
168879,1598470389,0.0,0.0,0.0,0.0,0.0,1.0,1


In [18]:
%sql select * from df_meses where numero_de_cliente in ('1359133970')

,numero_de_cliente,202101,202102,202103,202104,202105,202106,q_meses
0,1359133970,1.0,1.0,1.0,0.0,0.0,0.0,3


In [11]:
%sql select numero_de_cliente, foto_mes from df where numero_de_cliente = '1359133970' order by foto_mes

,numero_de_cliente,foto_mes
0,1359133970,202101
1,1359133970,202102
2,1359133970,202103


In [ ]:
# %%time
# %%sql 

# WITH mes_base AS (
# SELECT numero_de_cliente, foto_mes AS mes_base
# FROM df
# WHERE foto_mes = '202102')

# SELECT
# mb.numero_de_cliente,
# CASE

# WHEN NOT EXISTS (SELECT 1 FROM df p
# WHERE p.numero_de_cliente = mb.numero_de_cliente
# AND p.foto_mes = mb.mes_base + 1) THEN 'BAJA+1'
            
# WHEN NOT EXISTS (
# SELECT 1 FROM df p
# WHERE p.numero_de_cliente = mb.numero_de_cliente
# AND p.foto_mes = mb.mes_base + 2) THEN 'BAJA+2'

# ELSE 'CONTINUA'
# END AS clase_ternaria_202102
# FROM mes_base mb

In [ ]:
#Creo la clase_ternaria

duckdb.execute("""
DROP TABLE IF EXISTS clases;
CREATE TABLE clases AS
WITH max_mes AS (
    SELECT MAX(foto_mes) AS max_foto_mes
    FROM df
),
-- Agregar la conversión a periodo0 para cada registro
periodos AS (
    SELECT 
        numero_de_cliente,
        foto_mes,
        CAST(foto_mes / 100 AS INTEGER) * 12 + (foto_mes % 100) AS periodo0
    FROM df
),
max_periodo AS (
    SELECT MAX(periodo0) AS max_periodo0
    FROM periodos
)
SELECT
    d.numero_de_cliente,
    d.foto_mes,
    CASE
        WHEN d.periodo0 <= m.max_periodo0 - 1
             AND NOT EXISTS (
                SELECT 1
                FROM periodos p
                WHERE p.numero_de_cliente = d.numero_de_cliente
                  AND p.periodo0 = d.periodo0 + 1
             ) THEN 'BAJA+1'
               
        WHEN d.periodo0 <= m.max_periodo0 - 2
             AND NOT EXISTS (
                SELECT 1
                FROM periodos p
                WHERE p.numero_de_cliente = d.numero_de_cliente
                  AND p.periodo0 = d.periodo0 + 2
             ) THEN 'BAJA+2'
               
        ELSE 'CONTINUA'
    END AS clase_ternaria
FROM periodos d
CROSS JOIN max_periodo m;
""")

# Verificar que se creó
try:
    count_clases = duckdb.query("SELECT COUNT(*) FROM clases").fetchone()
    print(f"La tabla clases existe y tiene {count_clases[0]} registros")
except Exception as e:
    print(f"Error con tabla clases: {e}")
    print("La tabla clases no existe")


Tabla creada con 978439 registros


In [ ]:
# # Resumen por clase y mes
# query_simple = """
# SELECT 
#     clase_ternaria,
#     foto_mes,
#     COUNT(*) as cantidad_clientes
# FROM clases
# GROUP BY clase_ternaria, foto_mes
# ORDER BY clase_ternaria, foto_mes;
# """

# resultado = duckdb.query(query_simple).to_df()
# print(resultado)

# # Pivoteo la tabla para mostrarla
# import pandas as pd
# pivot_df = resultado.pivot(index='clase_ternaria', columns='foto_mes', values='cantidad_clientes').fillna(0)
# print(pivot_df)


   clase_ternaria  foto_mes  cantidad_clientes
0          BAJA+1    202101                622
1          BAJA+1    202102                831
2          BAJA+1    202103               1039
3          BAJA+1    202104                955
4          BAJA+1    202105               1134
5          BAJA+2    202101                825
6          BAJA+2    202102               1032
7          BAJA+2    202103                951
8          BAJA+2    202104               1130
9        CONTINUA    202101             160080
10       CONTINUA    202102             160292
11       CONTINUA    202103             161119
12       CONTINUA    202104             161333
13       CONTINUA    202105             162783
14       CONTINUA    202106             164313
foto_mes          202101    202102    202103    202104    202105    202106
clase_ternaria                                                            
BAJA+1             622.0     831.0    1039.0     955.0    1134.0       0.0
BAJA+2             825.

In [25]:
#Sumo clase_ternaria al dataframe original y lo guardo a un nuevo csv
df_final = duckdb.query("""
SELECT 
    df.*,
    c.clase_ternaria
FROM df
LEFT JOIN clases c
    ON df.numero_de_cliente = c.numero_de_cliente
   AND df.foto_mes = c.foto_mes
""").df() 


df_final.to_csv("C:/Users/Guada/Desktop/dmeyf/competencia_01_final.csv", index=False)


KeyboardInterrupt: 

In [ ]:
query_202101 = """
WITH mes_base AS (
    SELECT numero_de_cliente, foto_mes AS mes_base
    FROM df
    WHERE foto_mes = '202101'
)
SELECT
    mb.numero_de_cliente,
    CASE
        WHEN NOT EXISTS (
            SELECT 1 FROM df p
            WHERE p.numero_de_cliente = mb.numero_de_cliente
              AND p.foto_mes = mb.mes_base + 1
        ) THEN 'BAJA+1'
        WHEN NOT EXISTS (
            SELECT 1 FROM df p
            WHERE p.numero_de_cliente = mb.numero_de_cliente
              AND p.foto_mes = mb.mes_base + 2
        ) THEN 'BAJA+2'
        ELSE 'CONTINUA'
    END AS clase_ternaria_202101b
FROM mes_base mb
"""

df_clases_202101 = duckdb.query(query_202101).df()


In [126]:
df_clases_202101['clase_ternaria_202101'].value_counts()

clase_ternaria_202101
CONTINUA    160080
BAJA+2         825
BAJA+1         622
Name: count, dtype: int64

In [133]:
%%time
query_202102 = """
WITH mes_base AS (
    SELECT numero_de_cliente, foto_mes AS mes_base
    FROM df
    WHERE foto_mes = '202102'
)
SELECT
    mb.numero_de_cliente,
    CASE
        WHEN NOT EXISTS (
            SELECT 1 FROM df p
            WHERE p.numero_de_cliente = mb.numero_de_cliente
              AND p.foto_mes = mb.mes_base + 1
        ) THEN 'BAJA+1'
        WHEN NOT EXISTS (
            SELECT 1 FROM df p
            WHERE p.numero_de_cliente = mb.numero_de_cliente
              AND p.foto_mes = mb.mes_base + 2
        ) THEN 'BAJA+2'
        ELSE 'CONTINUA'
    END AS clase_ternaria_202102
FROM mes_base mb
"""

df_clases_202102 = duckdb.query(query_202102).df()


CPU times: total: 234 ms
Wall time: 142 ms


In [116]:
df_clases_202102['clase_ternaria_202102'].value_counts()

clase_ternaria_202102
CONTINUA    160292
BAJA+2        1032
BAJA+1         831
Name: count, dtype: int64

In [131]:
%%time
query_202104 = """
WITH mes_base AS (
    SELECT numero_de_cliente, foto_mes AS mes_base
    FROM df
    WHERE foto_mes = '202104')
SELECT
    mb.numero_de_cliente,
    CASE
        WHEN NOT EXISTS (
            SELECT 1 FROM df p
            WHERE p.numero_de_cliente = mb.numero_de_cliente
              AND p.foto_mes = mb.mes_base + 1
        ) THEN 'BAJA+1'
        WHEN NOT EXISTS (
            SELECT 1 FROM df p
            WHERE p.numero_de_cliente = mb.numero_de_cliente
              AND p.foto_mes = mb.mes_base + 2
        ) THEN 'BAJA+2'
        ELSE 'CONTINUA'
    END AS clase_ternaria_202104
FROM mes_base mb
"""

df_clases_202104 = duckdb.query(query_202104).df()


CPU times: total: 156 ms
Wall time: 109 ms


In [119]:
df_clases_202104

,numero_de_cliente,clase_ternaria_202104
0,249221323,CONTINUA
1,249227600,CONTINUA
2,249234235,CONTINUA
3,249244449,CONTINUA
4,249244739,CONTINUA
...,...,...
163413,1592258100,CONTINUA
163414,1592802499,CONTINUA
163415,1592805052,CONTINUA
163416,1592828238,CONTINUA


In [121]:
df_clases_202102['clase_ternaria_202102'].value_counts()

clase_ternaria_202102
CONTINUA    160292
BAJA+2        1032
BAJA+1         831
Name: count, dtype: int64

In [120]:
df_clases_202104['clase_ternaria_202104'].value_counts() 

clase_ternaria_202104
CONTINUA    161333
BAJA+2        1130
BAJA+1         955
Name: count, dtype: int64

In [ ]:
%sql

select 